<a href="https://colab.research.google.com/github/Bruno-svm/Bruno-svm/blob/main/tfc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math

# importando os dados da tabela do excel
# A configuração de importação foi feita em cima da tabela gerada pelo site da WEG, sem nenhuma edição anterior.


df = pd.read_excel("/content/30CV_premi.xlsx", header=1, skiprows=3)



In [ ]:
# Tabela antes do tratamento dos dados
df.head(8)

,Norma,Frequência,Tensão nominal,Polos,Potência,Carcaça,Unnamed: 6,Ip / In,Conjugado de partida,Conjugado máximo,...,Unnamed: 17,Unnamed: 18,Fator de Potência,Unnamed: 20,Unnamed: 21,Corrente nominal,Altitude,Regime,Temperatura Ambiente,Proteção
0,NaN,NaN,NaN,NaN,HP (cv),NaN,NaN,NaN,NaN,NaN,...,0.75,1.0,0.50,0.75,1.00,NaN,NaN,NaN,NaN,NaN
1,IEC,60 Hz,220/380/440V,4.0,30.00,180L,Carcaça Superior,8.0,3.4,3.2,...,93.30,93.6,0.66,0.76,0.81,76.2/44.1/38.1 A,1000 m,S1,-20ºC - +40ºC,IP55
2,IEC,60 Hz,220/380/440V,4.0,30.00,180M,Carcaça Padrão,8.0,3.4,3.2,...,93.50,93.6,0.66,0.76,0.81,76.2/44.1/38.1 A,1000 m,S1,-20ºC - +40ºC,IP55
3,IEC,60 Hz,220/380V,4.0,30.00,180L,Carcaça Superior,8.0,3.4,3.2,...,93.30,93.6,0.66,0.76,0.81,76.2/44.1 A,1000 m,S1,-20ºC - +40ºC,IP55
4,IEC,60 Hz,220/380V,4.0,30.00,180M,Carcaça Padrão,8.0,3.4,3.2,...,93.50,93.6,0.66,0.76,0.81,76.2/44.1 A,1000 m,S1,-20ºC - +40ºC,IP55
5,IEC,60 Hz,380/660V,4.0,30.00,180L,Carcaça Superior,8.0,3.4,3.2,...,93.30,93.6,0.66,0.76,0.81,44.1/25.4 A,1000 m,S1,-20ºC - +40ºC,IP55
6,IEC,60 Hz,380/660V,4.0,30.00,180M,Carcaça Padrão,8.0,3.4,3.2,...,93.50,93.6,0.66,0.76,0.81,44.1/25.4 A,1000 m,S1,-20ºC - +40ºC,IP55
7,IEC,60 Hz,440V,4.0,30.00,180L,Carcaça Superior,8.0,3.4,3.2,...,93.30,93.6,0.66,0.76,0.81,38.1 A,1000 m,S1,-20ºC - +40ºC,IP55


In [ ]:
# Excluindo algumas colunas que não serão usadas :
colunas_para_excluir = ['Unnamed: 6','Altitude','Unnamed: 21','Nível de ruído',
             'Temperatura Ambiente','Fator de serviço','Proteção',
             'Unnamed: 17','Unnamed: 18','Unnamed: 20','Regime']
df = df.drop(colunas_para_excluir, axis=1)

In [ ]:
# Retirando as linhas nulas da tabela
linhas_para_excluir = [0]
df = df.drop(linhas_para_excluir)

In [ ]:
!pip install openpyxl

In [ ]:
# Tabela apos a retirada das linhas nulas e colunas que não serão usadas
df.head()

,Norma,Frequência,Tensão nominal,Polos,Potência,Carcaça,Ip / In,Conjugado de partida,Conjugado máximo,Momento de inércia,Tempo de Rotor Bloqueado,Peso,Rotação nominal,Rendimento (%),Fator de Potência,Corrente nominal
1,IEC,60 Hz,220/380/440V,4.0,30.00,180L,8.0,3.4,3.2,0.19185 kgm2,11 s,176 kg,1775 rpm,92.4,0.66,76.2/44.1/38.1 A
2,IEC,60 Hz,220/380/440V,4.0,30.00,180M,8.0,3.4,3.2,0.19185 kgm2,20 s,184 kg,1770 rpm,93.0,0.66,76.2/44.1/38.1 A
3,IEC,60 Hz,220/380V,4.0,30.00,180L,8.0,3.4,3.2,0.19185 kgm2,11 s,176 kg,1775 rpm,92.4,0.66,76.2/44.1 A
4,IEC,60 Hz,220/380V,4.0,30.00,180M,8.0,3.4,3.2,0.19185 kgm2,20 s,184 kg,1770 rpm,93.0,0.66,76.2/44.1 A
5,IEC,60 Hz,380/660V,4.0,30.00,180L,8.0,3.4,3.2,0.19185 kgm2,11 s,176 kg,1775 rpm,92.4,0.66,44.1/25.4 A


In [ ]:
import re


# Definindo uma função para extrair apenas os números de uma string
def extrair_numeros(valor):
    numeros = re.findall(r'\d+\.\d+|\d+', str(valor))
    return ''.join(numeros)

# Aplicando a função à coluna 'Momento de inércia' para extrair apenas os números
df['Momento de inércia'] = df['Momento de inércia'].apply(extrair_numeros)

In [ ]:
import re


# Definindo uma função para extrair apenas os números de uma string
def extrair_numeros(valor):
    numeros = re.findall(r'\d+\.\d+|\d+', str(valor))
    return ''.join(numeros)

# Aplicando a função à coluna 'Rotação nominal' para extrair apenas os números
df['Rotação nominal'] = df['Rotação nominal'].apply(extrair_numeros)

In [ ]:
# Mudando o tipo de dado para float
df['Rotação nominal'] = df['Rotação nominal'].astype(float)

In [ ]:
# Mudando o tipo de dado para float
df['Potência'] = df['Potência'].astype(float)

In [ ]:
# Retirando possiveis dados nulos
df = df.dropna(subset=['Momento de inércia'])

In [ ]:
# mudando o tipo de dado para float
df['Momento de inércia'] = df['Momento de inércia'].astype(float)


In [ ]:
# formato dos dados da tabela
df.dtypes

Norma                        object
Frequência                   object
Tensão nominal               object
Polos                       float64
Potência                    float64
Carcaça                      object
Ip / In                     float64
Conjugado de partida        float64
Conjugado máximo            float64
Momento de inércia          float64
Tempo de Rotor Bloqueado     object
Peso                         object
Rotação nominal             float64
Rendimento (%)              float64
Fator de Potência           float64
Corrente nominal             object
dtype: object

In [ ]:
# Tabela pronta para a leitura do programa
df.head()

,Norma,Frequência,Tensão nominal,Polos,Potência,Carcaça,Ip / In,Conjugado de partida,Conjugado máximo,Momento de inércia,Tempo de Rotor Bloqueado,Peso,Rotação nominal,Rendimento (%),Fator de Potência,Corrente nominal
1,IEC,60 Hz,220/380/440V,4.0,30.0,180L,8.0,3.4,3.2,0.191852,11 s,176 kg,1775.0,92.4,0.66,76.2/44.1/38.1 A
2,IEC,60 Hz,220/380/440V,4.0,30.0,180M,8.0,3.4,3.2,0.191852,20 s,184 kg,1770.0,93.0,0.66,76.2/44.1/38.1 A
3,IEC,60 Hz,220/380V,4.0,30.0,180L,8.0,3.4,3.2,0.191852,11 s,176 kg,1775.0,92.4,0.66,76.2/44.1 A
4,IEC,60 Hz,220/380V,4.0,30.0,180M,8.0,3.4,3.2,0.191852,20 s,184 kg,1770.0,93.0,0.66,76.2/44.1 A
5,IEC,60 Hz,380/660V,4.0,30.0,180L,8.0,3.4,3.2,0.191852,11 s,176 kg,1775.0,92.4,0.66,44.1/25.4 A


In [ ]:
import pandas as pd
import math

def calcular_tempo_de_partida(potencia_Motor, inercia_Carga, torque_Inicial, torque_maximo,
                              inercia_motor, conjugado_carga, velocidade_motor):

    torque_nominal = (potencia_Motor*745.7)/(velocidade_motor*0.10471)
    conjugado_inicial_carga = (conjugado_carga / 5)
    conjugado_resistente = ((2 * conjugado_inicial_carga + conjugado_carga) / 3)
    conjugado_medio_motor = (0.45 * ((torque_Inicial/3) + (torque_maximo/3)) * torque_nominal )
    tempo_aceleracao = (2 * math.pi * (velocidade_motor/60) * (inercia_Carga + inercia_motor)) / (conjugado_medio_motor - conjugado_resistente)
    return tempo_aceleracao, torque_nominal

#Calculo realizado considerando 1/3 do torque

# Número total de linhas na tabela
total_linhas = len(df)

# Listas para armazenar os resultados
resultados_tempo_partida = []
resultados_torque_nominal = []

# Iterando sobre cada linha do DataFrame
for i in range(total_linhas):
    # Extrair os dados das colunas para a linha atual
    torque_Inicial = df.iloc[i, 7]
    torque_maximo = df.iloc[i, 8]
    inercia_motor = df.iloc[i, 9]
    conjugado_carga = 122.07  # Definido pela carga escolhida
    inercia_Carga = 0.292  # Definido pela carga escolhida
    potencia_Motor = df.iloc[i, 4]
    velocidade_motor = df.iloc[i, 12]

    # Calcular tempo de partida e torque nominal para a linha atual
    tempo_partida, torque_nominal = calcular_tempo_de_partida(potencia_Motor, inercia_Carga, torque_Inicial, torque_maximo,
                                                              inercia_motor, conjugado_carga, velocidade_motor)

    # Armazenar os resultados para a linha atual
    resultados_tempo_partida.append(tempo_partida)
    resultados_torque_nominal.append(torque_nominal)

# Exibindo os resultados
for i in range(total_linhas):
    print("Para o Motor", i+1, "- Tempo de partida é:", resultados_tempo_partida[i], "segundos - Torque nominal do Motor:", resultados_torque_nominal[i])

Para o Motor 1 - Tempo de partida é: 1.446052850416044 segundos - Torque nominal do Motor: 120.36462880040244
Para o Motor 2 - Tempo de partida é: 1.434217162906553 segundos - Torque nominal do Motor: 120.7046418761098
Para o Motor 3 - Tempo de partida é: 1.446052850416044 segundos - Torque nominal do Motor: 120.36462880040244
Para o Motor 4 - Tempo de partida é: 1.434217162906553 segundos - Torque nominal do Motor: 120.7046418761098
Para o Motor 5 - Tempo de partida é: 1.446052850416044 segundos - Torque nominal do Motor: 120.36462880040244
Para o Motor 6 - Tempo de partida é: 1.434217162906553 segundos - Torque nominal do Motor: 120.7046418761098
Para o Motor 7 - Tempo de partida é: 1.446052850416044 segundos - Torque nominal do Motor: 120.36462880040244
Para o Motor 8 - Tempo de partida é: 1.434217162906553 segundos - Torque nominal do Motor: 120.7046418761098
